# 1_PRREDICT BUILDING

# Import libraries

In [27]:
# basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# estimators


from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Classification Models
from sklearn.svm import LinearSVC
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# tools
from sklearn.model_selection import train_test_split

# regression model metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
# classification model metrics
from sklearn.metrics import accuracy_score

#Hyper parameter tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# To save the ML models
import joblib

# Import Data

In [2]:
path = '/home/ale/Dropbox/UBIQUM/4.DeepAnalytics&Visualization/T4M3.WiFiLocationing/Data/'
 
file = "0_DataPrepro.csv"

Tdata = pd.read_csv(path + file)

Tdata.head()




,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,...,100,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,100,100,-97,100,100,...,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807
4,100,100,100,100,100,100,100,100,100,100,...,100,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710


# Cascade Model

Also remember the cascade model thing. Predict first one thing, then another, then another etc.. in every iteration we add the features we want to predict. 

Steps
1. remove uselsss features
2. dummify
3. sample to iterate, but just to trial error 
4. divide train/test
5. apply the model
6. performace metrics

# 1. Predict Building

start by predicting the building beacuse it's the easiest one, just 3 possibilities: 0, 1, 2. 

- Predictors: all the WAP Features
- Predicted: BUILDINGid

## 1.1 Keep Usefull features

In [3]:
# make a copy of the dataset, keeping only the predictor/predicted features
Tdata_Building = Tdata.drop(columns = ['LONGITUDE', 'LATITUDE', 'FLOOR', 'SPACEID','RELATIVEPOSITION', 'USERID', 'PHONEID', 'TIMESTAMP'])

Tdata_Building.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520,BUILDINGID
0,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,1
1,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,1
2,100,100,100,100,100,100,100,-97,100,100,...,100,100,100,100,100,100,100,100,100,1
3,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,1
4,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,0


## 1.2 Dummify

In this case it's not needed, because i'm using all numerical variables to predict a categorical variable. 

## 1.3 Sample Data

As there is a lot of data, is a good idea to fail faster with a reduced proportion of the data

In [4]:
sample =  Tdata_Building

In [5]:
Tdata_Building = sample.sample(frac =.45, random_state= 200)

print("Len of the DF :" + str(len(Tdata_Building)))

#Tdata_Building.head()

Len of the DF :8972


## 1.4 X/y Split

In [6]:
X_building = Tdata_Building.iloc[:,0:520]
y_building = Tdata_Building.iloc[:,520:521]

X_building.head() # check predictors

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
998,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
2013,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1665,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
6188,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-83,-85,100,100,100
2671,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100


In [7]:
y_building.head() # check predicted

,BUILDINGID
998,1
2013,2
1665,2
6188,2
2671,2


## 1.5 Multicolinearity

For reference, the goal of regression is to isolate the relationship between each independent variable and the dependent variable. Multicollinearity weakens the statistical power of your model, thus leaving you unable to trust the p-values identifying which independent variables are statistically significant. In summary, multicollinearity won’t let you know the true effect of each variable.

### Calculate the Variance Inflation Factor (VIF).
VIF measures the collinearity among independent variables within a regression model. Then remove the multicorrelated features. 

https://kaiserm.medium.com/how-to-tackle-multicollinearity-79afe58e9479

In [8]:
from statsmodels.stats.outliers_influence import variance_inflation_factor


# Compute VIF
vif = pd.DataFrame()
vif["variables"] = X_building.columns
vif["VIF"] = [variance_inflation_factor(X_building.values, i) for i in range(X_building.shape[1])]

vif


/home/ale/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/ale/anaconda3/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,variables,VIF
0,WAP001,1.238410
1,WAP002,1.593780
2,WAP003,0.000000
3,WAP004,0.000000
4,WAP005,1.471728
...,...,...
515,WAP516,3.302083
516,WAP517,2.455898
517,WAP518,2.079820
518,WAP519,1.134106


### create a list with the features that exceeds the VIF treshold value
The [documentation](https://www.statsmodels.org/stable/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html) states that an independent variable is highly collinear with other independent variables when VIF > 5, and the parameter estimates will have large standard errors because of this.

In [9]:
count = 0
VIF_blackList = []

correlated_columns = []
for i in range(len(vif)):
    if vif.loc[i,"VIF"] >= 5:
        VIF_blackList.append(vif.loc[i,'variables'])
#        print(vif.loc[i,'variables'],vif.loc[i,'VIF'] )
        count += 1
print("Wap's with a VIF >= 5  -> " + str(count))

# uncomment to see the list of features to remove
#VIF_blackList

Wap's with a VIF >= 5  -> 85


### remove from the VIF_blackList

In [10]:
X_building = X_building.drop(columns = VIF_blackList)
X_building.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP509,WAP510,WAP511,WAP512,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
998,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
2013,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1665,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
6188,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-83,-85,100,100,100
2671,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100


## 1.6 Train/Test Split

In [11]:
# define X and y
X = X_building
y = y_building

# define train/test_ratio
train_ratio = 0.75
test_ratio = 0.25

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(1-train_ratio) )

# check dimensions
print('DF: ', Tdata_Building.shape)
print('X:', X.shape,'y:', y.shape)
print('X_train:', X_train.shape,'y_train:', y_train.shape)
print('X_test: ', X_test.shape,' y_test: ', y_test.shape)

DF:  (8972, 521)
X: (8972, 435) y: (8972, 1)
X_train: (6729, 435) y_train: (6729, 1)
X_test:  (2243, 435)  y_test:  (2243, 1)


## 1.6 Models

### 1.6.1 k-NN Classifier

also check radiusKnn, says it's better for large datasets https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.RadiusNeighborsClassifier.html#sklearn.neighbors.RadiusNeighborsClassifier

In [12]:
# define an empty dict to store the performance of different values of K
knn_performance = {}

for k in range(1,10):
    knn = KNeighborsClassifier(k, weights='distance')
    # train
    knn.fit(X_train, y_train.values.ravel())
    #get predictions
    knn_pred = knn.predict(X_test)
    # calculate the performance
    knn_accuracy = round(accuracy_score(y_test, knn_pred), 4)
    # store the performance in a dict
    knn_performance[k] = knn_accuracy

    
knn_performance_sorted = sorted(knn_performance.items(), key=lambda item:item[1], reverse=True)

# shows the full list sorted
#for i in knn_performance_sorted: print("k = " + str(i[0]) + " , accuracy: " + str(i[1]))

# shows only the best result
print("k = " + str(knn_performance_sorted[0][0]) + " , accuracy (on test set): " + str(knn_performance_sorted[0][1]))

k = 4 , accuracy (on test set): 0.9955


In [13]:
from sklearn.metrics import classification_report

classification_report(y_test, knn_pred)

'              precision    recall  f1-score   support\n\n           0       1.00      0.99      1.00       588\n           1       0.98      1.00      0.99       571\n           2       1.00      0.99      0.99      1084\n\n    accuracy                           0.99      2243\n   macro avg       0.99      0.99      0.99      2243\nweighted avg       0.99      0.99      0.99      2243\n'

In [14]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, knn_pred)

array([[ 585,    3,    0],
       [   0,  571,    0],
       [   0,   11, 1073]])

### save k-NN
In machine learning, while working with scikit learn library, we need to save the trained models in a file and restore them in order to reuse it to compare the model with other models, to test the model on a new data. The saving of data is called **Serialization**, while restoring the data is called **Deserialization**.
Also, we deal with different types and sizes of data. Some datasets are easily trained i.e- they take less time to train but the datasets whose size is large (more than 1GB) can take very large time to train on a local machine even with GPU. When we need the same trained data in some different project or later sometime, to avoid the wastage of the training time, store trained model so that it can be used anytime in the future. 
There are two ways we can save a model in scikit learn:
 
**Pickled** model as a file using joblib: Joblib is the replacement of pickle as it is more efficient on objects that carry large numpy arrays. These functions also accept file-like object instead of filenames.
     

- joblib.dump to serialize an object hierarchy 
- joblib.load to deserialize a data stream

In [36]:
path = '/home/ale/Dropbox/UBIQUM/4.DeepAnalytics&Visualization/T4M3.WiFiLocationing/'
file = "Building_KNN.pkl"
model = knn
 
# Save the model as a pickle in a file
joblib.dump(model, path+file)
 

['/home/ale/Dropbox/UBIQUM/4.DeepAnalytics&Visualization/T4M3.WiFiLocationing/Building_KNN.pkl']

### Load k-NN

In [35]:
# Load the model from the file
knn_from_joblib = joblib.load('/home/ale/Dropbox/UBIQUM/4.DeepAnalytics&Visualization/T4M3.WiFiLocationing/Building_KNN.pkl')
 
# Use the loaded model to make predictions
print(round(accuracy_score(y_test, knn_from_joblib.predict(X_test)), 4))


0.9938


### 1.6.2 Random Forest Classifier

In [17]:
RF = RandomForestClassifier()

# print the full list of parametrizable parameters
#RF.get_params(deep=True)


#Setup the parameters and distributions to sample from: param_dist

cv = 3
n_jobs = -1

# numers of trees
n_estimators = [2,5,8,10,12]
# max number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# max number of levels in tree
max_depth = [5, 10, 15, 20, 30, 40, 50]
#max_depth.append(None)
# min number of samples required to split a node
min_samples_split = [2,5,10,20,30]
# min number of samples required at each leaf node
min_samples_leaf = [2,5,10,15]


param_dist = {#'bootstrap': True, 
              #'ccp_alpha': 0.0,
              #'criterion': 'mse',
              'n_estimators': n_estimators, 
              'max_features': max_features,
              'max_depth':max_depth,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf
              #'max_leaf_nodes': None,
              #'max_samples': None,
              #'min_impurity_decrease': 0.0, 
              #'min_impurity_split': None, 
              #'min_weight_fraction_leaf': 0.0,           
              #'n_jobs': None, 
              #'oob_score': False, 
              #'random_state': None,
              #'verbose': 0, 
              #'warm_start': False
             }

# instantiate the RandomizedSearchCV object: model_cv
RF_RSCV1 = RandomizedSearchCV(RF, param_dist, cv=cv, n_jobs = n_jobs)

# Fit it to the data
RF_RSCV1.fit(X_train,y_train.values.ravel())

# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(RF_RSCV1.best_params_))

# evaulate a CV=3 using the parameters estimated with Randomized search
RF_result_CV1 = cross_val_score(RF_RSCV1, X_test, y_test.values.ravel(), cv=5, scoring='accuracy')

# prints all the cv=x results
#print(RF_result_CV1)

RF_accuracy = RF_RSCV1.score(X_test, y_test.values.ravel())
print (f'Random Forest Accuracy (on test set) : {RF_RSCV1.score(X_test, y_test.values.ravel()):.5f}')

Tuned Decision Tree Parameters: {'n_estimators': 5, 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 30}
Random Forest Accuracy (on test set) : 0.99287


### save RF

Joblib is an efficient object operator that carry large numpy arrays. These functions also accept file-like object instead of filenames.
     

    joblib.dump to serialize an object hierarchy 
    joblib.load to deserialize a data stream

In [37]:
path = '/home/ale/Dropbox/UBIQUM/4.DeepAnalytics&Visualization/T4M3.WiFiLocationing/'
file = 'Building_RF.pkl'
model = RF_RSCV1
 
# Save the model as a pickle in a file
joblib.dump(model, path+file)
 

['/home/ale/Dropbox/UBIQUM/4.DeepAnalytics&Visualization/T4M3.WiFiLocationing/Building_RF.pkl']

### Load k-NN

In [39]:
# Load the model from the file
RF_from_joblib = joblib.load('/home/ale/Dropbox/UBIQUM/4.DeepAnalytics&Visualization/T4M3.WiFiLocationing/Building_RF.pkl')

# Use the loaded model to make predictions
print(round(accuracy_score(y_test, RF_from_joblib.predict(X_test)), 4))


0.9929


### 1.6.3 Log Regression

In [21]:
model = LogisticRegression()

# print the full list of parametrizable parameters
# LogReg.get_params(deep=True)

#Setup the parameters and distributions to sample from: param_dist
cv = 3
n_jobs = -1 # uses all the processors
param_dist_LR = {#'C': 1.0,
              #'class_weight': None,
              #'dual': False,
              #'fit_intercept': True,
              #'intercept_scaling': 1,
              #'l1_ratio': None,
              'max_iter' : [600, 800, 1500, 3000],
              #'multi_class': 'auto',
              #'n_jobs': n_jobs,
              'penalty': ['l1', 'l2', 'none'], #'elasticnet', 
              #'random_state': random_state,
              'solver': ['sag', 'saga'],
              #'tol': 0.0001,
              #'verbose': 0,
              'warm_start': ["True", "False"]
}

LR_CV = RandomizedSearchCV(model, param_dist_LR, cv=cv, n_jobs=n_jobs)

# Fit it to the data
LR_CV.fit(X_train,y_train.values.ravel())

# Print the tuned parameters and score
print("Tuned Decision LogReg Parameters: {}".format(LR_CV.best_params_))



# evaulate a CV=x using the parameters estimated with Randomized search
LR_results_CV = cross_val_score(LR_CV, X_test, y_test.values.ravel(), cv=cv, scoring='accuracy')

RF_result_CV1_sorted = np.sort(RF_result_CV1)
print("Logistic Regression Accuracy CV (on test set):" + str(round(LR_results_CV[::-1][0], 4)))


/home/ale/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.99643335        nan 0.99643335 0.99643335 0.99643335 0.99643335
 0.99643335 0.99643335        nan 0.99643335]
  warnings.warn(
/home/ale/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Tuned Decision LogReg Parameters: {'warm_start': 'True', 'solver': 'sag', 'penalty': 'none', 'max_iter': 600}


/home/ale/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.99397724 0.9933079  0.99397724 0.99397724 0.9933079  0.99397724
 0.9933079  0.99397724 0.9933079         nan]
  warnings.warn(
/home/ale/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.99531996 0.9959893         nan 0.99465195 0.9959893  0.9959893
 0.9953213  0.9953213         nan 0.9959893 ]
  warnings.warn(
/home/ale/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.99531862 0.99464927 0.99598796        nan 0.99665596 0.99598796
 0.99531996 0.99531996 0.99598796 0.99598796]
  warnings.warn(


Logistic Regression Accuracy CV (on test set):0.9973


/home/ale/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [22]:
print(LR_Result)

NameError: name 'LR_Result' is not defined

In [ ]:
print (f'Accuracy - : {LR_CV.score(X,y):.3f}')

### save LR

Joblib is an efficient object operator that carry large numpy arrays. These functions also accept file-like object instead of filenames.
     

    joblib.dump to serialize an object hierarchy 
    joblib.load to deserialize a data stream

In [43]:
path = '/home/ale/Dropbox/UBIQUM/4.DeepAnalytics&Visualization/T4M3.WiFiLocationing/'
file = 'Building_LR.pkl'
model = LR_results_CV
 
# Save the model as a pickle in a file
joblib.dump(model, path+file)
 

['/home/ale/Dropbox/UBIQUM/4.DeepAnalytics&Visualization/T4M3.WiFiLocationing/Building_LR.pkl']

### Load k-NN

In [45]:
# Load the model from the file
LR_from_joblib = joblib.load('/home/ale/Dropbox/UBIQUM/4.DeepAnalytics&Visualization/T4M3.WiFiLocationing/Building_LR.pkl')

# Use the loaded model to make predictions
print(round(accuracy_score(y_test, LR_from_joblib.predict(X_test)), 4))


AttributeError: 'numpy.ndarray' object has no attribute 'predict'

# Save Data

In [ ]:
path = '/home/ale/Dropbox/UBIQUM/4.DeepAnalytics&Visualization/T4M3.WiFiLocationing/'

Tdata.to_csv(path + 'Data/0_DataPrepro.csv')

Tdata.head()